## Введение в обработку естественного языка

Домашнее задание №8

Урок 8. Рекуррентные нейронные сети RNN LSTM GRU

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

###Задание  
Данные берем отызывы за лето.

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи.

1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

**Установка библиотек**

In [1]:
!pip install pymorphy2
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 71.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=3f35c59c77f08b95cc3c754a8288d1ceb67cc1126e95cf1498096cc6d3574f1e
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=24969d413c4a9bcb9f665439a3dfd54d9ae5754ec767b41033c29ed8f00e48f0
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Succe

In [4]:
import pandas as pd
import numpy as np
import re
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec, FastText

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalMaxPool2D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils

from sklearn.metrics import classification_report

In [5]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
df = pd.read_excel('отзывы за лето (1).xls')

In [8]:
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [9]:
df = df.dropna()

In [10]:
df.shape

(20656, 3)

In [11]:
df.Rating.value_counts()

5    14584
1     2276
4     2137
3      911
2      748
Name: Rating, dtype: int64

In [12]:
# делаем колонку Content текстовой - так как ругается что - 'int' object has no attribute 'translate'

df.Content = df.Content.apply(lambda text: str(text))

<ipython-input-12-26a918284a53>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Content = df.Content.apply(lambda text: str(text))


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Content, df.Rating, test_size=0.3, random_state=42, stratify=df.Rating)

In [14]:
y_train.value_counts()

5    10209
1     1593
4     1496
3      638
2      523
Name: Rating, dtype: int64

In [15]:
y_test.value_counts()

5    4375
1     683
4     641
3     273
2     225
Name: Rating, dtype: int64

In [16]:
corpus_train = X_train
corpus_test = X_test

In [17]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(corpus_train)

sequences_train = tokenizer.texts_to_sequences(corpus_train)
sequences_test = tokenizer.texts_to_sequences(corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

## Построение сетей

In [18]:
num_classes = 6

epochs = 20
batch_size = 512

comparing_list = []

In [19]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

### CNN+pooling

In [20]:
net_name = 'CNN+pooling'

model = Sequential()
model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
# model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping],
                    )

Epoch 1/20
26/26 [==============================] - 16s 207ms/step - loss: 1.4695 - accuracy: 0.6520 - val_loss: 1.0992 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 6s 227ms/step - loss: 1.0165 - accuracy: 0.7064 - val_loss: 0.9385 - val_accuracy: 0.7026
Epoch 3/20
26/26 [==============================] - 5s 200ms/step - loss: 0.8346 - accuracy: 0.7285 - val_loss: 0.7622 - val_accuracy: 0.7573
Epoch 4/20
26/26 [==============================] - 5s 178ms/step - loss: 0.7012 - accuracy: 0.7737 - val_loss: 0.7010 - val_accuracy: 0.7725
Epoch 5/20
26/26 [==============================] - 6s 222ms/step - loss: 0.6150 - accuracy: 0.7947 - val_loss: 0.6630 - val_accuracy: 0.7801
Epoch 6/20
26/26 [==============================] - 4s 153ms/step - loss: 0.5373 - accuracy: 0.8249 - val_loss: 0.6482 - val_accuracy: 0.7822
Epoch 7/20
26/26 [==============================] - 5s 204ms/step - loss: 0.4715 - accuracy: 0.8478 - val_loss: 0.6515 - val_accuracy: 0.7822


In [21]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.39      0.92      0.55       683
           2       0.15      0.62      0.24       225
           3       0.13      0.52      0.21       273
           4       0.10      0.99      0.19       641
           5       0.82      0.97      0.89      4375

   micro avg       0.39      0.94      0.55      6197
   macro avg       0.27      0.67      0.35      6197
weighted avg       0.64      0.94      0.73      6197
 samples avg       0.42      0.94      0.57      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RNN

In [22]:
net_name = 'RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 14s 418ms/step - loss: 1.6076 - accuracy: 0.5031 - val_loss: 1.1737 - val_accuracy: 0.7248
Epoch 2/20
26/26 [==============================] - 11s 390ms/step - loss: 0.9438 - accuracy: 0.7370 - val_loss: 0.7575 - val_accuracy: 0.7593
Epoch 3/20
26/26 [==============================] - 15s 571ms/step - loss: 0.6752 - accuracy: 0.7840 - val_loss: 0.7028 - val_accuracy: 0.7690
Epoch 4/20
26/26 [==============================] - 9s 356ms/step - loss: 0.5325 - accuracy: 0.8221 - val_loss: 0.7111 - val_accuracy: 0.7649


In [23]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 4s 21ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.42      0.80      0.55       683
           2       0.14      0.42      0.21       225
           3       0.14      0.37      0.20       273
           4       0.10      0.97      0.19       641
           5       0.81      0.97      0.88      4375

   micro avg       0.40      0.91      0.56      6197
   macro avg       0.27      0.59      0.34      6197
weighted avg       0.64      0.91      0.72      6197
 samples avg       0.42      0.91      0.57      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CNN + RNN

In [24]:
net_name = 'CNN+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
# model.add(GlobalMaxPool1D(keepdims=True))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

# print(model.summary())
# plot_model(model, show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 11s 354ms/step - loss: 1.3624 - accuracy: 0.5299 - val_loss: 0.8754 - val_accuracy: 0.7248
Epoch 2/20
26/26 [==============================] - 9s 348ms/step - loss: 0.8907 - accuracy: 0.7257 - val_loss: 0.7712 - val_accuracy: 0.7344
Epoch 3/20
26/26 [==============================] - 8s 302ms/step - loss: 0.7455 - accuracy: 0.7673 - val_loss: 0.7089 - val_accuracy: 0.7559
Epoch 4/20
26/26 [==============================] - 9s 338ms/step - loss: 0.5983 - accuracy: 0.8062 - val_loss: 0.6805 - val_accuracy: 0.7669
Epoch 5/20
26/26 [==============================] - 9s 356ms/step - loss: 0.4624 - accuracy: 0.8548 - val_loss: 0.7035 - val_accuracy: 0.7573


In [25]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 3s 14ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.36      0.85      0.51       683
           2       0.13      0.35      0.19       225
           3       0.20      0.44      0.28       273
           4       0.10      0.98      0.19       641
           5       0.82      0.96      0.89      4375

   micro avg       0.40      0.91      0.56      6197
   macro avg       0.27      0.60      0.34      6197
weighted avg       0.64      0.91      0.72      6197
 samples avg       0.42      0.91      0.57      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CNN+pooling+RNN

In [26]:
net_name = 'CNN+pooling+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

# print(model.summary())
# plot_model(model, show_shapes=True, show_layer_names=True)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 8s 225ms/step - loss: 1.2713 - accuracy: 0.6858 - val_loss: 0.9567 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 4s 167ms/step - loss: 0.8759 - accuracy: 0.7151 - val_loss: 0.7323 - val_accuracy: 0.7531
Epoch 3/20
26/26 [==============================] - 5s 187ms/step - loss: 0.6994 - accuracy: 0.7650 - val_loss: 0.6641 - val_accuracy: 0.7746
Epoch 4/20
26/26 [==============================] - 5s 174ms/step - loss: 0.5975 - accuracy: 0.7930 - val_loss: 0.6458 - val_accuracy: 0.7759
Epoch 5/20
26/26 [==============================] - 3s 133ms/step - loss: 0.5188 - accuracy: 0.8213 - val_loss: 0.6631 - val_accuracy: 0.7732


In [27]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.41      0.89      0.56       683
           2       0.15      0.69      0.24       225
           3       0.15      0.67      0.25       273
           4       0.10      0.95      0.19       641
           5       0.83      0.97      0.89      4375

   micro avg       0.39      0.94      0.55      6197
   macro avg       0.27      0.69      0.35      6197
weighted avg       0.65      0.94      0.73      6197
 samples avg       0.42      0.94      0.58      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RNN + CNN + CNN

In [28]:
net_name = 'RNN+CNN+CNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(num_classes, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 16s 468ms/step - loss: 0.3932 - accuracy: 0.6739 - val_loss: 0.2471 - val_accuracy: 0.7165
Epoch 2/20
26/26 [==============================] - 11s 424ms/step - loss: 0.2332 - accuracy: 0.7290 - val_loss: 0.2194 - val_accuracy: 0.7344
Epoch 3/20
26/26 [==============================] - 11s 409ms/step - loss: 0.2005 - accuracy: 0.7629 - val_loss: 0.1974 - val_accuracy: 0.7669
Epoch 4/20
26/26 [==============================] - 12s 469ms/step - loss: 0.1692 - accuracy: 0.7917 - val_loss: 0.1970 - val_accuracy: 0.7524
Epoch 5/20
26/26 [==============================] - 11s 431ms/step - loss: 0.1487 - accuracy: 0.8195 - val_loss: 0.2040 - val_accuracy: 0.7580


In [29]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 5s 25ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.50      0.23      0.31       683
           2       0.00      0.00      0.00       225
           3       0.00      0.00      0.00       273
           4       0.15      0.00      0.01       641
           5       0.89      0.93      0.91      4375

   micro avg       0.86      0.68      0.76      6197
   macro avg       0.26      0.19      0.20      6197
weighted avg       0.70      0.68      0.68      6197
 samples avg       0.68      0.68      0.68      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RNN CNN pooling RNN

In [30]:
net_name = 'RNN+CNN+pooling+RNN'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True)) # keepdims Сохраняет размерность для SimpleRNN
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 15s 441ms/step - loss: 1.2846 - accuracy: 0.6683 - val_loss: 0.7773 - val_accuracy: 0.7497
Epoch 2/20
26/26 [==============================] - 11s 407ms/step - loss: 0.7424 - accuracy: 0.7477 - val_loss: 0.7023 - val_accuracy: 0.7573
Epoch 3/20
26/26 [==============================] - 10s 385ms/step - loss: 0.6366 - accuracy: 0.7793 - val_loss: 0.7009 - val_accuracy: 0.7663
Epoch 4/20
26/26 [==============================] - 9s 342ms/step - loss: 0.5511 - accuracy: 0.8033 - val_loss: 0.7159 - val_accuracy: 0.7711


In [31]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 5s 25ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.41      0.85      0.56       683
           2       0.15      0.48      0.23       225
           3       0.15      0.57      0.24       273
           4       0.10      0.95      0.19       641
           5       0.81      0.98      0.88      4375

   micro avg       0.40      0.92      0.56      6197
   macro avg       0.27      0.64      0.35      6197
weighted avg       0.64      0.92      0.72      6197
 samples avg       0.43      0.92      0.58      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## (CNN RNN)x3 + polling

In [32]:
net_name = '(CNN+RNN)x3+polling'

model = Sequential()

model.add(Embedding(input_dim=word_count,
                    output_dim=128,
                    input_length=training_length,
                    trainable=True,
                    mask_zero=True,
                    ))
# model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(64, 3, activation='relu'))
model.add(SimpleRNN(64, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(32, 2, activation='relu'))
model.add(SimpleRNN(32, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 24s 715ms/step - loss: 1.4231 - accuracy: 0.4743 - val_loss: 0.9486 - val_accuracy: 0.7026
Epoch 2/20
26/26 [==============================] - 17s 641ms/step - loss: 1.0378 - accuracy: 0.6786 - val_loss: 0.8106 - val_accuracy: 0.7427
Epoch 3/20
26/26 [==============================] - 16s 628ms/step - loss: 0.8905 - accuracy: 0.7210 - val_loss: 0.7344 - val_accuracy: 0.7566
Epoch 4/20
26/26 [==============================] - 18s 677ms/step - loss: 0.7823 - accuracy: 0.7489 - val_loss: 0.7075 - val_accuracy: 0.7586
Epoch 5/20
26/26 [==============================] - 16s 601ms/step - loss: 0.7059 - accuracy: 0.7700 - val_loss: 0.6874 - val_accuracy: 0.7628
Epoch 6/20
26/26 [==============================] - 15s 595ms/step - loss: 0.6633 - accuracy: 0.7772 - val_loss: 0.7000 - val_accuracy: 0.7545


In [33]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

comparing_list.append([net_name, history.history['accuracy'][-1], history.history['val_accuracy'][-1]])

194/194 [==============================] - 9s 43ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.31      0.95      0.47       683
           2       0.16      0.50      0.25       225
           3       0.02      0.32      0.03       273
           4       0.10      0.99      0.19       641
           5       0.79      0.98      0.88      4375

   micro avg       0.30      0.93      0.46      6197
   macro avg       0.23      0.62      0.30      6197
weighted avg       0.61      0.93      0.70      6197
 samples avg       0.31      0.93      0.46      6197



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Сравнение результатов

In [34]:
comparing_list

[['CNN+pooling', 0.8478444814682007, 0.7821576595306396],
 ['RNN', 0.8221009969711304, 0.7648686170578003],
 ['CNN+RNN', 0.8547606468200684, 0.7572613954544067],
 ['CNN+pooling+RNN', 0.8212556838989258, 0.7731673717498779],
 ['RNN+CNN+CNN', 0.8194882273674011, 0.7579529881477356],
 ['RNN+CNN+pooling+RNN', 0.8032736778259277, 0.7710926532745361],
 ['(CNN+RNN)x3+polling', 0.777222752571106, 0.7544951438903809]]

In [35]:
pd.DataFrame(comparing_list, columns=['net_name', 'accuracy', 'val_accuracy'])

,net_name,accuracy,val_accuracy
0,CNN+pooling,0.847844,0.782158
1,RNN,0.822101,0.764869
2,CNN+RNN,0.854761,0.757261
3,CNN+pooling+RNN,0.821256,0.773167
4,RNN+CNN+CNN,0.819488,0.757953
5,RNN+CNN+pooling+RNN,0.803274,0.771093
6,(CNN+RNN)x3+polling,0.777223,0.754495


Наиболее сбалансированный результат по выбранной метрике показала связка *CNN+pooling+RNN*, а также архитектура RNN без каких-либо излишеств 